In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

### Load Data

In [2]:
df = pd.read_excel('hedge_data.xls', sheet_name = [0,1,2], index_col = 0)
df = pd.concat(df.values(), axis=1).loc[:'1998-06',]
df.head()

,gross,net,Total Index,Convertible Arbitrage,Dedicated Short Bias,Emerging Markets,Equity Market Neutral,Event Driven,Event Driven Distressed,Event Driven Multi-Strategy,Event Driven Risk Arbitrage,Fixed Income Arbitrage,Global Macro,Long/Short Equity,Managed Futures,Multi-Strategy,Market Equity Index
date,,,,,,,,,,,,,,,,,
1994-04-30,0.0658,0.0508,-0.0196,-0.0274,0.0106,-0.0858,0.0003,-0.0088,-0.0093,-0.0079,-0.0118,-0.0042,-0.0181,-0.0178,0.0064,0.0236,0.0068
1994-05-31,-0.0415,-0.0315,0.0198,-0.0127,0.0200,-0.0098,-0.0036,-0.0040,-0.0109,0.0044,0.0000,0.0054,0.0355,0.0032,0.0051,-0.0372,0.0057
1994-06-30,0.1137,0.0817,-0.0104,-0.0002,0.0612,0.0024,0.0047,-0.0036,-0.0118,0.0060,-0.0005,-0.0063,-0.0115,-0.0254,0.0392,0.0338,-0.0303
1994-07-31,0.0355,0.0275,0.0010,-0.0011,-0.0143,0.0556,-0.0125,0.0029,0.0155,-0.0121,0.0032,-0.0043,-0.0060,0.0104,-0.0126,-0.0248,0.0281
1994-08-31,-0.0061,-0.0051,0.0256,-0.0025,-0.0587,0.1621,-0.0120,0.0061,0.0183,-0.0093,0.0148,-0.0030,0.0254,0.0143,-0.0048,-0.0345,0.0401


### Q1 Allocation

#### 1(a) Summary Statistics

In [3]:
funds_name = df.columns[2:-1]
funds = df[funds_name]
mean_ret_funds = funds.mean() * 12
vol_funds = funds.std() * np.sqrt(12)
SR_funds = mean_ret_funds / vol_funds
pd.DataFrame({'mean return': mean_ret_funds, 'volatility': vol_funds, 'Sharpe ratio':SR_funds})

,mean return,volatility,Sharpe ratio
Total Index,0.131929,0.084846,1.554931
Convertible Arbitrage,0.054118,0.035637,1.518564
Dedicated Short Bias,-0.055647,0.150176,-0.370545
Emerging Markets,0.031624,0.185129,0.170819
Equity Market Neutral,0.059224,0.037148,1.594258
Event Driven,0.094800,0.040418,2.345470
Event Driven Distressed,0.121929,0.054472,2.238380
Event Driven Multi-Strategy,0.077835,0.046771,1.664185
Event Driven Risk Arbitrage,0.056047,0.033577,1.669220
Fixed Income Arbitrage,0.043718,0.025507,1.713978


#### 1(b)

Event Driven has the best sharpe ratio, while Dedicated Short Bias has the worst sharpe ratio

#### 1(c)

Even if an asset has a bad Sharpe Ratio individually, it may have low correlation to the other securities such that it diversifies the overall risk. So while it may not have a great return-to-risk tradeoff individually, it can have a good return-to-risk tradeoff with respect to the marginal impact on the portfolio. 

#### 1(d)

In [4]:
funds.quantile(0.05)

Total Index                   -0.02160
Convertible Arbitrage         -0.01545
Dedicated Short Bias          -0.07010
Emerging Markets              -0.08435
Equity Market Neutral         -0.01230
Event Driven                  -0.01180
Event Driven Distressed       -0.01525
Event Driven Multi-Strategy   -0.01510
Event Driven Risk Arbitrage   -0.00990
Fixed Income Arbitrage        -0.01040
Global Macro                  -0.02775
Long/Short Equity             -0.02625
Managed Futures               -0.05625
Multi-Strategy                -0.02760
Name: 0.05, dtype: float64

#### 2(a)

In [5]:
def compute_mv(data, diag=False):
    mu_tilde = data.mean()
    covariance = data.cov()
    if diag:
        covariance = np.diag(np.diag(covariance))
    N = covariance.shape[0]
    cov_inv = np.linalg.inv(covariance)
    omega_tan = cov_inv @ mu_tilde /(np.ones(N) @ cov_inv @ mu_tilde)
    return pd.Series(omega_tan,index=data.columns)

omega_tan = compute_mv(funds)
omega_tan

Total Index                   -0.607763
Convertible Arbitrage         -0.111832
Dedicated Short Bias           0.101699
Emerging Markets               0.004128
Equity Market Neutral          0.140937
Event Driven                  -0.829420
Event Driven Distressed        0.787013
Event Driven Multi-Strategy    0.214434
Event Driven Risk Arbitrage    0.486110
Fixed Income Arbitrage         0.387043
Global Macro                   0.375383
Long/Short Equity              0.094926
Managed Futures               -0.085109
Multi-Strategy                 0.042451
dtype: float64

#### 3(a) 

In [6]:
omega_tan_diag = compute_mv(funds, True)
omega_tan_diag

Total Index                    0.045529
Convertible Arbitrage          0.105860
Dedicated Short Bias          -0.006130
Emerging Markets               0.002292
Equity Market Neutral          0.106617
Event Driven                   0.144163
Event Driven Distressed        0.102085
Event Driven Multi-Strategy    0.088395
Event Driven Risk Arbitrage    0.123503
Fixed Income Arbitrage         0.166939
Global Macro                   0.026764
Long/Short Equity              0.039463
Managed Futures               -0.000642
Multi-Strategy                 0.055162
dtype: float64

#### 3(b)

Based on HW#1 and class discussion, we expect the diagonalized version to do better out-of-sample. Though it is a biased estimate (we are purposely using the wrong covariance matrix!) this is worth higher statistical precision. If we use the full covariance matrix in the tangency formula, it is an input to the formula which is very imprecisely estimated, then inverted to make it even more imprecise, then we get extremely imprecise outputs. 

#### 3(c)

Different objective function (care about quantiles not variance?) different constraints (limits to long-short holdings), etc. Also fine if they talk about non-normality of returns or non iid returns.

### Q2 Performance

#### 1(a)  

see df

#### 2(a) 

In [7]:
y = df['net']
X = sm.add_constant(df['Total Index'])
res = sm.OLS(y,X,missing='drop').fit()
alpha, beta = res.params
rsquared = res.rsquared
print(f'alpha is {alpha:.4f}, beta is {beta:.4f}, and R^2 is {rsquared:.4f}')

alpha is 0.0135, beta is 0.0497, and R^2 is 0.0014


c:\users\shu\appdata\local\programs\python\python37\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


#### 2(b) 

No. The intercept significantly above 0, indicating existence of other factors.

#### 2(c)

No. The $R^2$ is very small, and only 0.14% of the variation is explained by the Total Index

#### 2(d)

If the Total Index goes up by 1%, we would expect return of LTCM goes uo by 0.0497% that month

#### 3(a)

In [8]:
resid_vol = res.resid.std()
IR = alpha / resid_vol
print(f'IR is {IR:.4f}')

IR is 0.4144


#### 3(b)

The Info Ratio tells us the Sharpe Ratio of the LTCM return relative to the benchmark. (So any discussion of it as a relative mean per relative risk tradeoff—doesn’t is OK)

### Q3 Pricing

#### 1

alpha = 0 (i.e. insignificant). Mean return of LTCM can be perfectly explained by the Total Index, and no intercept left

beta = $\frac{Cov(r_{LTCM}, r_{total\_index})}{Var(r_{total\_index})}$

r-squared can be any value. The pricing model does not account for the variation of returns. It's a model reflecting relationship between mean returns. 

#### 2

Two approaches on this question, largely depending on whether you read this as a comparison to the ESTIMATED tangency which you computed before or to the TRUE (theoretical) tangency which itself works as a perfect LFPM.

One good answer: The tangency portfolio is the perfect LFPM by construction First Theorem of Asset Pricing. Thus, if the Total Index works as the LFPM, then the Total Index must be the (correctly estimated) tangency portfolio. Thus, they are the same SR.

Another good answer: If the Total Index is the perfect LFPM, then we know that expected excess return is only earned by having beta (and correlation) to it. Volatility from any other source is not adding expected mean return. Thus, the most efficient mean-per-vol one can get is through the factor itself. (In the notes we show SR of anything is SR of the perfect factor multiplied by the correlation between the two.) Thus, the Total Index has the max SR ratio and will be higher than the SR of the estimated version of the tangency portfolio. (It will be equal to the theoretical tangency portfolio, per the other argument above.)
